# OCLC Retained Holdings Lookup — Aggregated Version (Google Colab)

This notebook:
- Uploads a CSV of OCLC numbers (header: `oclcNumber`)
- Gets an OAuth token using WSKey & Secret
- Calls the retained holdings endpoint
- Aggregates results so each OCN has one row with: OCN, Title, All Institution Symbols, All Institution Names
- Saves and downloads a CSV of aggregated results.

## 1) Install & Imports

In [ ]:
!pip -q install requests pandas
import requests, pandas as pd, io
from google.colab import files
print('Libraries ready.')


## 2) Configure your OCLC credentials

In [ ]:
WSKEY = '[Enter Your OCLC WSKEY Here for the MetadataAPI]'
SECRET = '[Enter your OCLC Secret Here for the MetadataAPI]'
SCOPE = 'WorldCatMetadataAPI'
TOKEN_URL = 'https://oauth.oclc.org/token'
API_URL = 'https://metadata.api.oclc.org/worldcat/search/bibs-retained-holdings'
print('Config ready.')


## 3) Upload your CSV of OCLC numbers

In [ ]:
uploaded = files.upload()
if not uploaded:
    raise RuntimeError('No file uploaded.')
file_name = next(iter(uploaded.keys()))
oclc_df = pd.read_csv(io.BytesIO(uploaded[file_name]))
if 'oclcNumber' not in oclc_df.columns:
    raise ValueError('CSV must have column oclcNumber')
oclc_numbers = oclc_df['oclcNumber'].dropna().astype(str).tolist()
print(f'Found {len(oclc_numbers)} OCLC numbers.')
oclc_df.head()


## 4) Get OAuth token

In [ ]:
def get_token(wskey, secret, scope='WorldCatMetadataAPI'):
    resp = requests.post(TOKEN_URL, auth=(wskey, secret), headers={'Content-Type': 'application/x-www-form-urlencoded'}, data={'grant_type': 'client_credentials', 'scope': scope})
    resp.raise_for_status()
    return resp.json()['access_token']
token = get_token(WSKEY, SECRET, SCOPE)
print('Token acquired.')


## 5) Query API and aggregate results

In [ ]:
headers = {'Authorization': f'Bearer {token}', 'Accept': 'application/json'}
aggregated = []
for i, ocn in enumerate(oclc_numbers, start=1):
    print(f'[{i}/{len(oclc_numbers)}] OCN {ocn} ...')
    r = requests.get(API_URL, headers=headers, params={'oclcNumber': ocn})
    if r.status_code != 200:
        aggregated.append({'oclcNumber': ocn, 'title': '', 'allSymbols': '', 'allNames': '', 'notes': f'Error {r.status_code}'})
        continue
    data = r.json()
    brief_records = data.get('briefRecords', [])
    if not brief_records:
        aggregated.append({'oclcNumber': ocn, 'title': '', 'allSymbols': '', 'allNames': '', 'notes': 'No holdings'})
        continue
    # Assume one title per OCN
    title = brief_records[0].get('title', '')
    symbols = []
    names = []
    for br in brief_records:
        inst = br.get('institutionHolding', {})
        if isinstance(inst, dict):
            for ih in inst.get('briefHoldings', []):
                symbols.append(ih.get('oclcSymbol', ''))
                names.append(ih.get('institutionName', ''))
        elif isinstance(inst, list):
            for ih in inst:
                symbols.append(ih.get('oclcSymbol', ih.get('symbol', '')))
                names.append(ih.get('institutionName', ih.get('name', '')))
    aggregated.append({'oclcNumber': ocn, 'title': title, 'allSymbols': '|'.join(filter(None, symbols)), 'allNames': ' | '.join(filter(None, names))})
print('Aggregation complete.')


## 6) Save and download aggregated results

In [ ]:
out_df = pd.DataFrame(aggregated)
out_name = 'retained_holdings_aggregated.csv'
out_df.to_csv(out_name, index=False)
print(f'Saved: {out_name}')
files.download(out_name)
out_df.head()
